In [60]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

covid_dataset = pd.read_csv('google_new.csv', na_values=['NA'])

#Análise dos dados 

#Procura de valores nulos 
print(covid_dataset.isnull().sum())
#Não há valores em falta no dataset

iso                  0
country              0
date                 0
grocery_pharmacy     0
parks                0
residential          0
retail_recreation    0
transit_stations     0
workplaces           0
total_cases          0
fatalities           0
dtype: int64


In [61]:
#Análise dos dados 

#distribuição por classes, distribuição de valores em cada atributo, etc

#dividir os dados por países
countries = covid_dataset.groupby('country')

#média dos valores de cada país
mean =countries[['grocery_pharmacy','parks', 'retail_recreation',
                'transit_stations', 'workplaces', 'residential']].mean() 

print(mean)

                grocery_pharmacy      parks  retail_recreation  \
country                                                          
Argentina             -19.020047 -42.277535         -37.566070   
Australia               6.003023 -14.828302         -13.599047   
Brazil                 -7.304302 -28.471791         -29.426140   
Canada                 -4.026442   1.619884         -22.450279   
France                -18.395791 -28.106698         -43.221163   
Germany                -3.408930  10.049000         -27.155419   
India                 -21.119977 -18.746047         -29.776116   
Indonesia              -4.432512 -16.480302         -14.131442   
Italy                 -28.045977 -47.853186         -56.008000   
Japan                   3.558907   3.742140          -7.026093   
Mexico                 -3.511651 -14.220884         -16.682698   
Saudi Arabia           -9.152860 -25.963279         -27.978372   
South Africa           -8.987070 -18.496605         -19.657116   
South Kore

In [62]:
#Iso e Country são variáveis redundantes
covid_dataset.drop('iso', axis=1, inplace=True)



In [66]:
#Adicionar os casos recuperados (dados retirados deste repositório https://github.com/imdevskp/covid_19_jhu_data_web_scrap_and_cleaning)

recv_df = pd.read_csv('time_series_covid19_recovered_global.csv')
recv_df.drop('Province/State', axis=1, inplace=True)
dates = recovered.columns[4:]
recovered=recv_df.groupby(['Country/Region'], as_index=False)[dates].sum()

covid_dataset['date']= pd.to_datetime(covid_dataset.date)

covid_dataset['date'] = covid_dataset['date'].dt.strftime('%m/%d/%Y')

recv_df_long = recovered.melt(id_vars=['Country/Region'],value_vars=dates, var_name='date', value_name='Recovered')
recv_df_long= recv_df_long.rename(columns={'Country/Region': 'country'})

frames = [covid_dataset, recv_df_long]

full_table = pd.merge(left=covid_dataset, right=recv_df_long, on=['country', 'date']) #problema: South Korea desaparece, porque não tem na lista dos recuperados

print(full_table)

#full_table.to_csv("result.csv", index = False)
 


          country        date  grocery_pharmacy   parks  residential  \
0       Argentina  02/23/2020             8.185  17.563        0.437   
1       Argentina  02/24/2020           -15.875  25.605        6.321   
2       Argentina  02/25/2020           -17.135   1.325        6.895   
3       Argentina  02/26/2020             2.304   5.399       -2.273   
4       Argentina  02/27/2020            -0.404  -5.267       -0.493   
..            ...         ...               ...     ...          ...   
769  South Africa  04/01/2020           -42.446 -37.600       34.916   
770  South Africa  04/02/2020           -40.523 -38.226       36.550   
771  South Africa  04/03/2020           -42.107 -43.271       38.379   
772  South Africa  04/04/2020           -51.515 -57.252       26.499   
773  South Africa  04/05/2020           -51.420 -53.273       22.351   

     retail_recreation  transit_stations  workplaces  total_cases  fatalities  \
0               13.644             5.334      -2.486  

In [67]:
#Mais exemplos de dados a adicionar
#Active: Active cases = total confirmed - total recovered - total deaths.
#Case-Fatality Ratio (%): = confirmed cases per 100,000 persons.
#Testing Rate: = total test results per 100,000 persons. 
#Hospitalization Rate (%): = Total number hospitalized / Number confirmed cases.

#https://scikit-learn.org/stable/modules/preprocessing.html
from sklearn import preprocessing
preprocessing.scale(covid_dataset[['grocery_pharmacy','parks', 'retail_recreation',
                'transit_stations', 'workplaces', 'residential'] ])

array([[ 0.77061857,  0.94175912,  1.28654914,  1.11630666,  0.64081792,
        -0.86430523],
       [-0.35894183,  1.18640874,  0.52406489,  0.05250048, -1.33094526,
        -0.30599381],
       [-0.41809587,  0.44777498,  0.20947596, -0.00380484, -1.19099902,
        -0.25152903],
       ...,
       [-1.59047253, -0.90890175, -1.46277854, -1.64192526, -2.06487355,
         2.73587357],
       [-2.032156  , -1.3342246 , -1.62665922, -1.61152038, -1.38516769,
         1.60862345],
       [-2.02769598, -1.21317749, -1.60483429, -1.64381314, -0.97056636,
         1.21503477]])

          country        date  grocery_pharmacy   parks  residential  \
0       Argentina  02/23/2020             8.185  17.563        0.437   
1       Argentina  02/24/2020           -15.875  25.605        6.321   
2       Argentina  02/25/2020           -17.135   1.325        6.895   
3       Argentina  02/26/2020             2.304   5.399       -2.273   
4       Argentina  02/27/2020            -0.404  -5.267       -0.493   
..            ...         ...               ...     ...          ...   
769  South Africa  04/01/2020           -42.446 -37.600       34.916   
770  South Africa  04/02/2020           -40.523 -38.226       36.550   
771  South Africa  04/03/2020           -42.107 -43.271       38.379   
772  South Africa  04/04/2020           -51.515 -57.252       26.499   
773  South Africa  04/05/2020           -51.420 -53.273       22.351   

     retail_recreation  transit_stations  workplaces  total_cases  fatalities  \
0               13.644             5.334      -2.486  